In [1]:
import serial
import numpy as np

arduino_port = "COM5"
baud = 9600 
fileName="Data.csv"
ser = serial.Serial(arduino_port, baud)
file = open(fileName, "a")

In [2]:
del_t=100
SoC = 1;
V_ct = 0.007;
V_df = 0.004;
tau_ct = 4;
tau_df = 100;
R_ct = 0.001;
R_df = 0.01;
R_0 = 0.2;
R_l = 2.2;
Charge = 0.6*3600;


x = np.array([[SoC],
              [V_ct],
              [V_df]]);

A = np.array([[1, 0, 0],
             [0, np.exp(-(del_t/1000)/tau_ct), 0],
             [0, 0, np.exp(-(del_t/1000)/tau_df)]]);
              
B = np.array([[-(del_t/1000)/Charge],
              [R_ct*(1-np.exp(-(del_t/1000)/tau_ct))],
              [R_df*(1-np.exp(-(del_t/1000)/tau_df))]]);

P = np.array([[1e-3, 0, 0],
              [0, 1e-6, 0],
              [0, 0, 1e-6]]);

R = np.array([1e-3]);

Q = np.array([[0, 0, 0],
              [0, 1e-4, 0],
           
              [0, 0, 1e-5]]);

C = np.array([1, -1, -1]);

In [3]:
SoC = 1
I = 0
x[0][0]=1
while x[0][0] > 0.01:
    getData=str(ser.readline())[2:][:-5]
    I = getData[5:][:]
    I = float(I)
    
    #19a
    x_pred = np.dot(A, x)+(B*I)
    
    #19b
    P_pred = np.dot(np.dot(A, P), np.transpose(A)) + Q
       
    #Parameter Update   
    C[0] = (float(getData[0:][:-5]))/x_pred[0][0]
                                
    #20a
    L = np.dot(P_pred, np.transpose(C))/(np.dot(C, np.dot(P_pred, np.transpose(C)))+R)
    
    #20b
    x = x_pred + (L*(I*R_l - np.dot(C, x_pred) + I*R_0))
    
    #20c
    P = np.dot((np.identity(3, dtype = float) - np.dot(L, C)), P_pred)
    
    SoC = x[0][0]
    
    file = open(fileName, "a")
    file.write(getData[0:][:-5]+",")
    file.write(str(SoC))
    file.write("\n")
file.close()

In [4]:
SoC = 1
I = 0
x[0][0]=1
while x[0][0] > 0.01:
    getData=str(ser.readline())[2:][:-5]
    I = getData[5:][:]
    I = float(I)
    
    #19a
    x_pred = np.dot(A, x)+(B*I)
    
    #19b
    P_pred = np.dot(np.dot(A, P), np.transpose(A)) + Q
    
    H = np.array([1, 1/x_pred[0][0], -x_pred[0][0], np.log(x_pred[0][0]), np.log(1-x_pred[0][0])])
       
    C[0] = np.dot(H, (float(getData[0:][:-5]))*np.dot(np.linalg.inv(np.dot(np.transpose(H), H)), np.transpose(H)))/(float(getData[0:][:-5]))
    
                              
    #20a
    L = np.dot(P_pred, np.transpose(C))/(np.dot(C, np.dot(P_pred, np.transpose(C)))+R)
    
    #20b
    x = x_pred + (L*(I*R_l - np.dot(C, x_pred) + I*R_0))
    
    #20c
    P = np.dot((np.identity(3, dtype = float) - np.dot(L, C)), P_pred)
    
    SoC = x[0][0]
    
    file = open(fileName, "a")
    file.write(getData[0:][:-5]+",")
    file.write(str(SoC))
    file.write("\n")

file.close()

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [5]:
I = 0
x[0][0]=1
while x[0][0] > 0.01:
    getData=str(ser.readline())[2:][:-5]
    print(getData)
    I = getData[5:][:]
    I = float(I)
    
    #19a
    x_pred = np.dot(A, x)+(B*I)
    
    #19b
    P_pred = np.dot(np.dot(A, P), np.transpose(A)) + Q
    
    #C[0] = (3149.3 - (288.24/x_pred[0][0]) - (0.5748*x_pred[0][0]) + (54.8063*np.log(x_pred[0][0]))- (29.833*np.log(1-x_pred[0][0])))/(2*x_pred[0][0]);
  
    #20a
    L = np.dot(P_pred, np.transpose(C))/(np.dot(C, np.dot(P_pred, np.transpose(C)))+R)
    
    #20b
    x = x_pred + (L*(I*R_l - np.dot(C, x_pred) + I*R_0))
    
    #20c
    P = np.dot((np.identity(3, dtype = float) - np.dot(L, C)), P_pred)
    
    
    file = open(fileName, "a")
    file.write(getData[0:][:-5]+",")
    file.write(str(x[0][0]))
    file.write("\n")

file.close()

1.43,0.30
1.44,0.29
